In [1]:
default_dict = Dict(
    :r => 10,
    :epsilon => 0.25,
    :kappa => 1.6,
    :delta => 1/3,
    :opacity => 0.65,
    :theta_frac => 1.0,
    :theta_cut => 0.0,
)

function tokamak_c(cur_dict=default_dict)
    term_0 = 0.3 * ( 1 - cur_dict[:delta] ^ 2 )
    term_1 = ( 1 - 2 * cur_dict[:delta] - term_0 ) / 8
    term_2 = cur_dict[:delta] / 2
    
    cur_c = [
        ( 0 - term_2 ),
        ( 1 + term_1 ),
        ( 0 + term_2 ),
        ( 0 - term_1 )
    ]
    
    cur_c
end

function tokamak_u(cur_phi, cur_dict=default_dict)
    cur_u = 0.0
    for (cur_index, cur_c) in enumerate(tokamak_c(cur_dict))
        cur_u += cur_c * cos( cur_phi * (cur_index-1) )
    end
    cur_u
end

tokamak_v(cur_phi, cur_dict=default_dict) = cur_dict[:kappa] * sin(cur_phi)

tokamak_r(cur_phi, cur_dict=default_dict) = cur_dict[:r] * ( 1 + cur_dict[:epsilon] * tokamak_u(cur_phi, cur_dict) )
tokamak_z(cur_phi, cur_dict=default_dict) = cur_dict[:r] * ( 0 + cur_dict[:epsilon] * tokamak_v(cur_phi, cur_dict) )

tokamak_x(cur_phi, cur_theta, cur_dict=default_dict) = tokamak_r(cur_phi, cur_dict) * cos(cur_theta)
tokamak_y(cur_phi, cur_theta, cur_dict=default_dict) = tokamak_r(cur_phi, cur_dict) * sin(cur_theta)

tokamak_z(cur_phi, cur_theta::AbstractFloat, cur_dict=default_dict) = tokamak_z(cur_phi, cur_dict)


tokamak_z (generic function with 4 methods)

In [2]:
using Plots
pgfplots()

using DataStructures
return

In [89]:
cur_dicts = OrderedDict(
    "Tokamak" => deepcopy(default_dict),
    "Plasma" => deepcopy(default_dict)
)

cur_dicts["Tokamak"][:theta_frac] = 3/4
cur_dicts["Tokamak"][:theta_cut] = 1/4

cur_dicts["Plasma"][:epsilon] = 0.1
cur_dicts["Plasma"][:kappa] = 1.3
cur_dicts["Plasma"][:opacity] = 0.45

cur_plot = plot()

for (cur_index, (cur_key, tmp_dict)) in enumerate(cur_dicts)
    theta_count = 4 * (5) + 1
    phi_count = 4 * (3) + 1
    
    d_theta = 2 * pi / ( theta_count - 1 )
    d_phi = 2 * pi / ( phi_count - 1 )
    
    for cur_j in 1:Int( ( theta_count - 1 ) * tmp_dict[:theta_frac] )
        if iszero(tmp_dict[:theta_cut]) || tmp_dict[:theta_cut] < (cur_j/theta_count) < ( 1 - tmp_dict[:theta_cut] )
            cur_phi_list = 1:( phi_count - 1 )
        else
            cur_phi_list = ( Int( ( phi_count - 1 ) / 2 ) + 1 ):( phi_count - 1 ) 
        end

        for cur_i in cur_phi_list
            cur_xx = []
            cur_yy = []
            cur_zz = []

            prev_phi = ( cur_i - 1 ) * d_phi
            prev_theta = ( cur_j - 1 ) * d_theta

            next_phi = prev_phi + d_phi
            next_theta = prev_theta + d_theta        
            
            push!(cur_xx, tokamak_x(prev_phi, prev_theta, tmp_dict))
            push!(cur_yy, tokamak_y(prev_phi, prev_theta, tmp_dict))
            push!(cur_zz, tokamak_z(prev_phi, prev_theta, tmp_dict))
            push!(cur_xx, tokamak_x(prev_phi, next_theta, tmp_dict))
            push!(cur_yy, tokamak_y(prev_phi, next_theta, tmp_dict))
            push!(cur_zz, tokamak_z(prev_phi, next_theta, tmp_dict))
            push!(cur_xx, tokamak_x(next_phi, next_theta, tmp_dict))
            push!(cur_yy, tokamak_y(next_phi, next_theta, tmp_dict))
            push!(cur_zz, tokamak_z(next_phi, next_theta, tmp_dict))
            push!(cur_xx, tokamak_x(next_phi, prev_theta, tmp_dict))
            push!(cur_yy, tokamak_y(next_phi, prev_theta, tmp_dict))
            push!(cur_zz, tokamak_z(next_phi, prev_theta, tmp_dict))
            push!(cur_xx, tokamak_x(prev_phi, prev_theta, tmp_dict))
            push!(cur_yy, tokamak_y(prev_phi, prev_theta, tmp_dict))
            push!(cur_zz, tokamak_z(prev_phi, prev_theta, tmp_dict))
            
            fill_opacity = tmp_dict[:opacity]
            lineopacity = (2-cur_index)
            
            if cur_key == "Plasma"
                if max(prev_theta, next_theta) > pi/2 && min(prev_theta, next_theta) < 3*pi/2
                    cur_far_point = pi - pi/6
                    mid_theta = middle(prev_theta, next_theta)
                    if mid_theta > pi
                        cur_rel_dist = abs( cur_far_point - min(prev_theta, next_theta) )
                        cur_rel_dist /= abs( cur_far_point - 3*pi/2 )
                    else
                        cur_rel_dist = abs( cur_far_point - max(prev_theta, next_theta) )
                        cur_rel_dist /= abs( cur_far_point - pi/2 )
                    end
                    fill_opacity *= 0.35 + 0.65 * (cur_rel_dist) ^ 2.5
                end
            end

            plot!(cur_xx, cur_yy, cur_zz, fill=true, fillopacity=fill_opacity, lineopacity=0.2*lineopacity, color=cur_index, width=0, label="")
        end
    end
end

plot!(aspectratio=:equal, xticks=false, yticks=false, zticks=false)

nn = 31

tt = linspace(0, 2*pi, nn)
zz = linspace(-1, +1, nn)

mm = 2.375
pp = -pi + pi/32

xx = cos.(mm*tt + pp)
yy = sin.(mm*tt + pp)

for cur_index in 1:nn-1
    cur_xx = []
    cur_yy = []
    cur_zz = []
    
    push!(cur_xx, 1.0*xx[cur_index])
    push!(cur_yy, 1.0*yy[cur_index])
    push!(cur_zz, 1.0*zz[cur_index])
    
    push!(cur_xx, 1.0*xx[cur_index+1])
    push!(cur_yy, 1.0*yy[cur_index+1])
    push!(cur_zz, 1.0*zz[cur_index+1])
    
    push!(cur_xx, 0.65*xx[cur_index+1])
    push!(cur_yy, 0.65*yy[cur_index+1])
    push!(cur_zz, 1.0*zz[cur_index+1])
    
    push!(cur_xx, 0.65*xx[cur_index])
    push!(cur_yy, 0.65*yy[cur_index])
    push!(cur_zz, 1.0*zz[cur_index])
    
    push!(cur_xx, 1.0*xx[cur_index])
    push!(cur_yy, 1.0*yy[cur_index])
    push!(cur_zz, 1.0*zz[cur_index])
    
    cur_xx .*= 1.05 * 3
    cur_yy .*= 1.05 * 3
    cur_zz .*= 0.8 * 3.5
    
    plot!(cur_xx,cur_yy,cur_zz,fill=true,color=3, label="", fillopacity=0.7, lineopacity=0, width=0, grid=false)
end

plot!([0],[0],[0], color=1, opacity=0.6, label="Tokamak")
plot!([0],[0],[0], color=2, opacity=0.6, label="Plasma")
plot!([0],[0],[0], color=3, opacity=0.6, label="Solenoid", ticks=false)

title!("A Magnetic Fusion Reactor")


<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 7.140625 0 L 7.140625 -0.3125 L 6.96875 -0.3125 C 6.375 -0.3125 6.234375 -0.375 6.125 -0.703125 L 3.96875 -6.9375 C 3.921875 -7.0625 3.890625 -7.140625 3.734375 -7.140625 C 3.578125 -7.140625 3.546875 -7.078125 3.5 -6.9375 L 1.4375 -0.984375 C 1.25 -0.46875 0.859375 -0.3125 0.3125 -0.3125 L 0.3125 0 L 1.328125 -0.03125 L 2.484375 0 L 2.484375 -0.3125 C 1.984375 -0.3125 1.734375 -0.5625 1.734375 -0.8125 C 1.734375 -0.84375 1.75 -0.953125 1.75 -0.96875 L 2.21875 -2.265625 L 4.671875 -2.265625 L 5.203125 -0.75 C 5.21875 -0.703125 5.234375 -0.640625 5.234375 -0.609375 C 5.234375 -0.3125 4.671875 -0.3125 4.40625 -0.3125 L 4.40625 0 C 4.765625 -0.03125 5.46875 -0.03125 5.84375 -0.03125 Z M 4.5625 -2.578125 L 2.328125 -2.578125 L 3.4375 -5.828125 Z M 4.5625 -2.578125 "/>
 
 
<path style="stroke:none;" d="M 8.765625 0 L 8.765625 -0.3125 L 8.515625 -0.3125 C 7.75 -0.3125 7.734375 -0.421875 7.734375 -0.78125 L 7.734375 -6.03125 C 7.734375 -6.390625 7.75 -6.5 8.515625 -6.5 L 8.765625 -6.5 L 8.765625 -6.8125 L 7.078125 -6.8125 C 6.8125 -6.8125 6.8125 -6.796875 6.75 -6.625 L 4.5625 -1 L 2.40625 -6.59375 C 2.3125 -6.8125 2.28125 -6.8125 2.046875 -6.8125 L 0.375 -6.8125 L 0.375 -6.5 L 0.609375 -6.5 C 1.375 -6.5 1.390625 -6.390625 1.390625 -6.03125 L 1.390625 -1.046875 C 1.390625 -0.78125 1.390625 -0.3125 0.375 -0.3125 L 0.375 0 L 1.53125 -0.03125 L 2.703125 0 L 2.703125 -0.3125 C 1.671875 -0.3125 1.671875 -0.78125 1.671875 -1.046875 L 1.671875 -6.421875 L 1.6875 -6.421875 L 4.09375 -0.21875 C 4.140625 -0.09375 4.1875 0 4.28125 0 C 4.390625 0 4.421875 -0.078125 4.46875 -0.1875 L 6.921875 -6.5 L 6.921875 -0.78125 C 6.921875 -0.421875 6.90625 -0.3125 6.140625 -0.3125 L 5.90625 -0.3125 L 5.90625 0 C 6.265625 -0.03125 6.953125 -0.03125 7.34375 -0.03125 C 7.71875 -0.03125 8.390625 -0.03125 8.765625 0 Z M 8.765625 0 "/>
 
 
<path style="stroke:none;" d="M 4.8125 -0.890625 L 4.8125 -1.453125 L 4.5625 -1.453125 L 4.5625 -0.890625 C 4.5625 -0.3125 4.3125 -0.25 4.203125 -0.25 C 3.875 -0.25 3.84375 -0.703125 3.84375 -0.75 L 3.84375 -2.734375 C 3.84375 -3.15625 3.84375 -3.546875 3.484375 -3.921875 C 3.09375 -4.3125 2.59375 -4.46875 2.109375 -4.46875 C 1.296875 -4.46875 0.609375 -4 0.609375 -3.34375 C 0.609375 -3.046875 0.8125 -2.875 1.0625 -2.875 C 1.34375 -2.875 1.53125 -3.078125 1.53125 -3.328125 C 1.53125 -3.453125 1.46875 -3.78125 1.015625 -3.78125 C 1.28125 -4.140625 1.78125 -4.25 2.09375 -4.25 C 2.578125 -4.25 3.15625 -3.859375 3.15625 -2.96875 L 3.15625 -2.609375 C 2.640625 -2.578125 1.9375 -2.546875 1.3125 -2.25 C 0.5625 -1.90625 0.3125 -1.390625 0.3125 -0.953125 C 0.3125 -0.140625 1.28125 0.109375 1.90625 0.109375 C 2.578125 0.109375 3.03125 -0.296875 3.21875 -0.75 C 3.265625 -0.359375 3.53125 0.0625 4 0.0625 C 4.203125 0.0625 4.8125 -0.078125 4.8125 -0.890625 Z M 3.15625 -1.390625 C 3.15625 -0.453125 2.4375 -0.109375 1.984375 -0.109375 C 1.5 -0.109375 1.09375 -0.453125 1.09375 -0.953125 C 1.09375 -1.5 1.5 -2.328125 3.15625 -2.390625 Z M 3.15625 -1.390625 "/>
 
 
<path style="stroke:none;" d="M 4.828125 -4.03125 C 4.828125 -4.203125 4.71875 -4.515625 4.328125 -4.515625 C 4.125 -4.515625 3.6875 -4.453125 3.265625 -4.046875 C 2.84375 -4.375 2.4375 -4.40625 2.21875 -4.40625 C 1.28125 -4.40625 0.59375 -3.71875 0.59375 -2.953125 C 0.59375 -2.515625 0.8125 -2.140625 1.0625 -1.921875 C 0.9375 -1.78125 0.75 -1.453125 0.75 -1.09375 C 0.75 -0.78125 0.890625 -0.40625 1.203125 -0.203125 C 0.59375 -0.046875 0.28125 0.390625 0.28125 0.78125 C 0.28125 1.5 1.265625 2.046875 2.484375 2.046875 C 3.65625 2.046875 4.6875 1.546875 4.6875 0.765625 C 4.6875 0.421875 4.5625 -0.09375 4.046875 -0.375 C 3.515625 -0.640625 2.9375 -0.640625 2.328125 -0.640625 C 2.078125 -0.640625 1.65625 -0.640625 1.578125 -0.65625 C 1.265625 -0.703125 1.0625 -1 1.0625 -1.328125 C 1.0625 -1.359375 1.0625 -1.59375 1.21875 -1.796875 C 1.609375 -1.515625 2.03125 -1.484375 2.21875 -1.484375 C 3.140625 -1.484375

In [90]:
savefig(cur_plot, "fusion_reactor.tex")